In [1]:
# Open the file in read mode
with open('slides.txt', 'r') as file:
    # Read the entire contents of the file
    content = file.read()
    # Print the content
    print(content)


**Slide 1: Title**
- Title: Understanding Climate Change
- Subtitle: A Global Challenge and Opportunity

**Slide 2: Introduction to Climate Change**
- Definition of Climate Change
- Causes: Greenhouse gas emissions, deforestation, industrialization

**Slide 3: Evidence of Climate Change**
- Temperature rise: Historical data and projections
- Extreme weather events: Hurricanes, floods, droughts
- Melting ice caps and rising sea levels

**Slide 4: Impact on Ecosystems**
- Loss of biodiversity
- Disruption of ecosystems
- Threats to wildlife habitats

**Slide 5: Effects on Human Health**
- Heat-related illnesses
- Spread of infectious diseases
- Food and water insecurity

**Slide 6: Economic Implications**
- Costs of adaptation and mitigation
- Disruption of industries: Agriculture, tourism, fisheries
- Increased healthcare expenses

**Slide 7: Social Justice and Equity**
- Disproportionate impact on vulnerable communities
- Environmental racism
- Importance of equitable solutions

**Slid

In [4]:
import json

In [5]:
with open("slides.json", 'r') as file:
    data = json.load(file)

In [6]:
data

{'Understanding Climate Change': 'Climate change is a long-term shift in global or regional climate patterns. It is primarily due to human activities such as burning fossil fuels, deforestation, and industrial processes.',
 'Causes of Climate Change': 'The main causes of climate change include greenhouse gas emissions (carbon dioxide, methane, nitrous oxide), deforestation, industrialization, and agricultural practices.',
 'Effects of Climate Change': 'Climate change leads to a range of impacts including rising global temperatures, more frequent and severe weather events (such as hurricanes, floods, and droughts), melting ice caps and glaciers, and rising sea levels.',
 'Impact on Ecosystems': 'Climate change disrupts ecosystems, leading to loss of biodiversity, habitat destruction, shifts in species distribution, and increased risks of extinction for many plants and animals.',
 'Impact on Human Health': 'Climate change poses significant risks to human health, including heat-related il

In [7]:
from pptx import Presentation

# Create a presentation object
prs = Presentation()

In [8]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

def add_image_and_text_slide(prs, image_path, title, content):
    # Choose a slide layout with content and picture placeholder
    slide_layout = prs.slide_layouts[5]  # 5 corresponds to 'Title Slide with Picture Placeholder'

    # Add the new slide to the presentation
    slide = prs.slides.add_slide(slide_layout)

    # Set title
    title_shape = slide.shapes.title
    title_shape.text = title

    # Add image on the left side
    left_inch = Inches(0.5)
    top_inch = Inches(1.5)
    pic = slide.shapes.add_picture(image_path, left_inch, top_inch, width=Inches(4))

    # Add text on the right side
    text_box = slide.shapes.add_textbox(Inches(5), top_inch, Inches(3), Inches(5))
    text_frame = text_box.text_frame
    p = text_frame.add_paragraph()
    p.text = content
    p.alignment = PP_ALIGN.LEFT
    p.font.size = Pt(18)


In [1]:
from pptx import Presentation

def read_pptx(filename):
    """
    Read a PowerPoint presentation (.pptx) file and return the presentation object.

    Parameters:
    - filename (str): The name of the PowerPoint (.pptx) file to read.

    Returns:
    - prs (Presentation): The presentation object.
    """
    # Create a presentation object
    prs = Presentation(filename)

    return prs

In [10]:
fname = "slidego_template.pptx"
prs = read_pptx(fname)

In [11]:
# Add a slide with image on the left and text on the right
add_image_and_text_slide(prs, 'earth.jpg', 'Slide Title', 'Text content goes here.')
# Save the presentation
prs.save("slidego_pres.pptx")

In [12]:
[slide_layout.name for slide_layout in prs.slide_layouts]

['TITLE',
 'SECTION_HEADER',
 'TITLE_AND_BODY',
 'TITLE_AND_TWO_COLUMNS',
 'TITLE_ONLY',
 'ONE_COLUMN_TEXT',
 'MAIN_POINT',
 'SECTION_TITLE_AND_DESCRIPTION',
 'CAPTION_ONLY',
 'BIG_NUMBER',
 'BLANK',
 'BLANK_1_1_1_1_1_1',
 'CUSTOM_10',
 'BLANK_1_1',
 'CUSTOM',
 'CUSTOM_11',
 'CUSTOM_4',
 'CUSTOM_4_1',
 'TITLE_AND_TWO_COLUMNS_1',
 'TITLE_AND_TWO_COLUMNS_1_2',
 'TITLE_AND_TWO_COLUMNS_1_1',
 'TITLE_AND_TWO_COLUMNS_1_1_1',
 'CUSTOM_6',
 'CUSTOM_6_1',
 'CUSTOM_5',
 'CUSTOM_7',
 'CUSTOM_8',
 'CUSTOM_3_1',
 'CUSTOM_9',
 'CUSTOM_9_1']